<a href="https://colab.research.google.com/github/adrcynical01/DLI_Assignment/blob/main/DLI_Assignment_Official.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.0 Importing the Libraries